In [25]:

def beta1(fc_prime, unidades="psi"):
    if unidades == "psi":
        return __beta1_psi(fc_prime)
    elif unidades == "MPa":
        return __beta1_mpa(fc_prime)
    else:
        raise ValueError("Unidades inválidas. Debe ser 'psi' o 'MPa'.")
    
def __beta1_psi(fc_prime):
    """Calcula β1 en unidades de lb/plg²."""
    if fc_prime <= 4000:
        return 0.85
    else:
        beta1 = 0.85 - (fc_prime - 4000) * 0.00005
        return max(beta1, 0.65)  # β1 no puede ser menor a 0.65

def __beta1_mpa(fc_prime):
    """Calcula β1 en unidades de MPa."""
    if fc_prime <= 30:
        return 0.85
    else:
        beta1 = 0.85 - 0.008 * (fc_prime - 30)
        return max(beta1, 0.65)  # β1 no puede ser menor a 0.65


# Para lb/plg²
fc_prime_psi = 4500  # resistencia a compresión en lb/plg²
print(beta1(fc_prime_psi, "psi"))  # Resultado: valor de β1

# Para MPa
fc_prime_mpa = 35  # resistencia a compresión en MPa
print(beta1(fc_prime_mpa, "MPa"))  # Resultado: valor de β1

0.825
0.8099999999999999


In [145]:

def peralte_efectivo(h, r):
    return h - r

def cuantia_de_acero(As, b, d):
    p = As / (b * d)
    return round(p, 5)

def altura_bloque(As, fc_prime, fy, b):
    a = As * fy / (0.85 * fc_prime * b)
    return round(a, 2)

def peralte_eje_neutro(a, B1):
    c = a / B1
    return round(c, 2)

from math import sqrt

def Amin(fc_prime, fy, b, d, unidades="psi"):
    if unidades == "psi":
        return __Amin_psi(fc_prime, fy, b, d)
    elif unidades == "MPa":
        return __Amin_mpa(fc_prime, fy, b, d)
    else:
        raise ValueError("Unidades inválidas. Debe ser 'psi' o 'MPa'.")

def __Amin_psi(fc_prime, fy, b, d):
    Amin_1 = 3 * sqrt(fc_prime) * b * d / fy
    Amin_2 = 200 * b * d /fy 
    return max(Amin_1, Amin_2)

def __Amin_mpa(fc_prime, fy, b, d):
    Amin_1 = sqrt(fc_prime) * b * d / (4 * fy)
    Amin_2 = 1.4 * b * d /fy 
    return max(Amin_1, Amin_2)

def cuantia_de_acero_min(fc_prime, fy, unidades="psi"):
    if unidades == "psi":
        return round(__pmin_psi(fc_prime, fy), 5)
    elif unidades == "MPa":
        return round(__pmin_mpa(fc_prime, fy), 5)
    else:
        raise ValueError("Unidades inválidas. Debe ser 'psi' o 'MPa'.")

def __pmin_psi(fc_prime, fy):
    pmin_1 = 3 * sqrt(fc_prime) / fy
    pmin_2 = 200 /fy 
    return max(pmin_1, pmin_2)

def __pmin_mpa(fc_prime, fy):
    pmin_1 = sqrt(fc_prime) / (4 * fy)
    pmin_2 = 1.4 /fy 
    return max(pmin_1, pmin_2)


def cuantia_de_acero_balanceada(B1, fc_prime, fy, unidades="psi"):
    if unidades == "psi":
        return round(__pb_psi(B1, fc_prime, fy), 5)
    elif unidades == "MPa":
        return round(__pb_mpa(B1, fc_prime, fy), 5)
    else:
        raise ValueError("Unidades inválidas. Debe ser 'psi' o 'MPa'.")
    
def __pb_psi(B1, fc_prime, fy):
    return (0.85 * B1 * fc_prime / fy) * (87000 / (87000 + fy))

def __pb_mpa(B1, fc_prime, fy):
    return (0.85 * B1 * fc_prime / fy) * (600 / (600 + fy))


def cuantia_de_acero_max(pb):
    return round(0.75 * pb, 5)

def deformacion_unitaria(d, c):
    eu = round(0.003 * (d - c) /c, 5)
    if eu < 0.004:
        print("Controlada por Compresión")
        print("violación del código… demasiado acero")
        phi = None
    elif eu >= 0.005:
        print("Controlada por Tensión")
        phi = 0.90
    else:
        print("Zona de Transición")
        phi = round(0.65 + (eu - 0.002) * 0.25 / (0.005 - 0.002), 3)
    return eu, phi

def momentos(phi, As, fy, d, a, unidades):
    if unidades == "psi":
        Fact = 12 * 1000
    elif unidades == "MPa":
        Fact = 1000 * 1000
    else:
        raise ValueError("Unidades inválidas. Debe ser 'psi' o 'MPa'.")
    if phi==None:
        return None, None
    else:
        Momento = As * fy * (d - 0.5 * a) / Fact
        phiM = phi * Momento
        return round(Momento, 3), round(phiM, 3)

In [146]:
fc_prime = 4000
fy = 60000
unidades = "psi"
As = 4
b = 15
h = 27
r = 3


d = peralte_efectivo(h, r)
print(f"d = {d}")
p = cuantia_de_acero(As, b, d)
print(f"p = {p}")
pmin = cuantia_de_acero_min(fc_prime, fy, unidades)
print(f"pmin = {pmin}")
B1 = beta1(fc_prime, unidades)
print(f"B1 = {B1}")
pb = cuantia_de_acero_balanceada(B1, fc_prime, fy, unidades)
print(f"pb = {pb}")
pmax = cuantia_de_acero_max(pb)
print(f"pmax = {pmax}")
a = altura_bloque(As, fc_prime, fy, b)
print(f"a = {a}")
c = peralte_eje_neutro(a, B1)
print(f"c = {c}")
et, phi = deformacion_unitaria(d, c)
print(f"et = {et} | phi = {phi}")
M, QM = momentos(phi, As, fy, d, a, unidades)
print(f"M = {M} | QM = {QM}")

d = 24
p = 0.01111
pmin = 0.00333
B1 = 0.85
pb = 0.02851
pmax = 0.02138
a = 4.71
c = 5.54
Controlada por Tensión
et = 0.01 | phi = 0.9
M = 432.9 | QM = 389.61


In [147]:
fc_prime = 4000
fy = 60000
unidades = "psi"
As = 4.68
b = 12
h = 18
r = 3


d = peralte_efectivo(h, r)
print(f"d = {d}")
p = cuantia_de_acero(As, b, d)
print(f"p = {p}")
pmin = cuantia_de_acero_min(fc_prime, fy, unidades)
print(f"pmin = {pmin}")
B1 = beta1(fc_prime, unidades)
print(f"B1 = {B1}")
pb = cuantia_de_acero_balanceada(B1, fc_prime, fy, unidades)
print(f"pb = {pb}")
pmax = cuantia_de_acero_max(pb)
print(f"pmax = {pmax}")
a = altura_bloque(As, fc_prime, fy, b)
print(f"a = {a}")
c = peralte_eje_neutro(a, B1)
print(f"c = {c}")
et, phi = deformacion_unitaria(d, c)
print(f"et = {et} | phi = {phi}")
M, QM = momentos(phi, As, fy, d, a, unidades)
print(f"M = {M} | QM = {QM}")

d = 15
p = 0.026
pmin = 0.00333
B1 = 0.85
pb = 0.02851
pmax = 0.02138
a = 6.88
c = 8.09
Controlada por Compresión
violación del código… demasiado acero
et = 0.00256 | phi = None
M = None | QM = None


In [148]:
fc_prime = 4000
fy = 60000
unidades = "psi"
As = 3
b = 10
h = 18
r = 3


d = peralte_efectivo(h, r)
print(f"d = {d}")
p = cuantia_de_acero(As, b, d)
print(f"p = {p}")
pmin = cuantia_de_acero_min(fc_prime, fy, unidades)
print(f"pmin = {pmin}")
B1 = beta1(fc_prime, unidades)
print(f"B1 = {B1}")
pb = cuantia_de_acero_balanceada(B1, fc_prime, fy, unidades)
print(f"pb = {pb}")
pmax = cuantia_de_acero_max(pb)
print(f"pmax = {pmax}")
a = altura_bloque(As, fc_prime, fy, b)
print(f"a = {a}")
c = peralte_eje_neutro(a, B1)
print(f"c = {c}")
et, phi = deformacion_unitaria(d, c)
print(f"et = {et} | phi = {phi}")
M, QM = momentos(phi, As, fy, d, a, unidades)
print(f"M = {M} | QM = {QM}")

d = 15
p = 0.02
pmin = 0.00333
B1 = 0.85
pb = 0.02851
pmax = 0.02138
a = 5.29
c = 6.22
Zona de Transición
et = 0.00423 | phi = 0.836
M = 185.325 | QM = 154.932


In [149]:
fc_prime = 28
fy = 420
unidades = "MPa"
As = 3276
b = 350
h = 700
r = 75


d = peralte_efectivo(h, r)
print(f"d = {d}")
p = cuantia_de_acero(As, b, d)
print(f"p = {p}")
pmin = cuantia_de_acero_min(fc_prime, fy, unidades)
print(f"pmin = {pmin}")
B1 = beta1(fc_prime, unidades)
print(f"B1 = {B1}")
pb = cuantia_de_acero_balanceada(B1, fc_prime, fy, unidades)
print(f"pb = {pb}")
pmax = cuantia_de_acero_max(pb)
print(f"pmax = {pmax}")
a = altura_bloque(As, fc_prime, fy, b)
print(f"a = {a}")
c = peralte_eje_neutro(a, B1)
print(f"c = {c}")
et, phi = deformacion_unitaria(d, c)
print(f"et = {et} | phi = {phi}")
M, QM = momentos(phi, As, fy, d, a, unidades)
print(f"M = {M} | QM = {QM}")

d = 625
p = 0.01498
pmin = 0.00333
B1 = 0.85
pb = 0.02833
pmax = 0.02125
a = 165.18
c = 194.33
Controlada por Tensión
et = 0.00665 | phi = 0.9
M = 746.313 | QM = 671.681
